In [5]:
%pip install pyarrow fastparquet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 15.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 685.4/685.4 kB 14.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 15.7 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd

In [8]:
import os
os.chdir("..")
!pwd

/Users/emre/Projects/EvalLM


In [9]:
df = pd.read_parquet("excluded/arena/arenadatasi.parquet")
df.head()

,question_id,model_a,model_b,winner,judge,conversation_a,conversation_b,turn,anony,language,tstamp,openai_moderation,toxic_chat_tag
0,58210e39b3fd4441a2bd4a518bb44c2d,chatglm-6b,koala-13b,model_b,arena_user_973,[{'content': 'What is the difference between O...,[{'content': 'What is the difference between O...,1,True,English,1.682352e+09,"{'categories': {'harassment': False, 'harassme...","{'roberta-large': {'flagged': False, 'probabil..."
1,2564acd09e3942fd97657d05282d4389,oasst-pythia-12b,alpaca-13b,tie,arena_user_973,[{'content': 'Why did my parent not invite me ...,[{'content': 'Why did my parent not invite me ...,1,True,English,1.682352e+09,"{'categories': {'harassment': False, 'harassme...","{'roberta-large': {'flagged': False, 'probabil..."
2,90bfd142157948aba01931726c888e7f,koala-13b,oasst-pythia-12b,model_b,arena_user_973,"[{'content': 'Fuji vs. Nikon, which is better?...","[{'content': 'Fuji vs. Nikon, which is better?...",1,True,English,1.682352e+09,"{'categories': {'harassment': False, 'harassme...","{'roberta-large': {'flagged': False, 'probabil..."
3,a7c5accc53e649a3bc6b2e41d962ebc4,vicuna-13b,oasst-pythia-12b,model_b,arena_user_973,[{'content': 'How to build an arena for chatbo...,[{'content': 'How to build an arena for chatbo...,1,True,English,1.682352e+09,"{'categories': {'harassment': False, 'harassme...","{'roberta-large': {'flagged': False, 'probabil..."
4,adf27e819a3c494cb6e993f0c660e097,vicuna-13b,koala-13b,model_a,arena_user_973,"[{'content': 'When is it today?', 'role': 'use...","[{'content': 'When is it today?', 'role': 'use...",1,True,English,1.682352e+09,"{'categories': {'harassment': False, 'harassme...","{'roberta-large': {'flagged': False, 'probabil..."


In [13]:
is_toxic = df["toxic_chat_tag"].apply(lambda x: x['roberta-large']['flagged'])
df["is_toxic"] = is_toxic
df.head(), df.shape

(                        question_id           model_a           model_b  \
 0  58210e39b3fd4441a2bd4a518bb44c2d        chatglm-6b         koala-13b   
 1  2564acd09e3942fd97657d05282d4389  oasst-pythia-12b        alpaca-13b   
 2  90bfd142157948aba01931726c888e7f         koala-13b  oasst-pythia-12b   
 3  a7c5accc53e649a3bc6b2e41d962ebc4        vicuna-13b  oasst-pythia-12b   
 4  adf27e819a3c494cb6e993f0c660e097        vicuna-13b         koala-13b   
 
     winner           judge                                     conversation_a  \
 0  model_b  arena_user_973  [{'content': 'What is the difference between O...   
 1      tie  arena_user_973  [{'content': 'Why did my parent not invite me ...   
 2  model_b  arena_user_973  [{'content': 'Fuji vs. Nikon, which is better?...   
 3  model_b  arena_user_973  [{'content': 'How to build an arena for chatbo...   
 4  model_a  arena_user_973  [{'content': 'When is it today?', 'role': 'use...   
 
                                       conversat

In [15]:
df = df[df["is_toxic"] == False]
df.shape

(32482, 14)

In [16]:
df = df[df["language"] == "English"]
df.shape

(28731, 14)

In [17]:
df = df[["model_a", "model_b", "winner", "conversation_a", "conversation_b"]]
df.head()

,model_a,model_b,winner,conversation_a,conversation_b
0,chatglm-6b,koala-13b,model_b,[{'content': 'What is the difference between O...,[{'content': 'What is the difference between O...
1,oasst-pythia-12b,alpaca-13b,tie,[{'content': 'Why did my parent not invite me ...,[{'content': 'Why did my parent not invite me ...
2,koala-13b,oasst-pythia-12b,model_b,"[{'content': 'Fuji vs. Nikon, which is better?...","[{'content': 'Fuji vs. Nikon, which is better?..."
3,vicuna-13b,oasst-pythia-12b,model_b,[{'content': 'How to build an arena for chatbo...,[{'content': 'How to build an arena for chatbo...
4,vicuna-13b,koala-13b,model_a,"[{'content': 'When is it today?', 'role': 'use...","[{'content': 'When is it today?', 'role': 'use..."


In [18]:
def extract_question(row):
    if (row[0]["role"] == "user" and row[0]["content"] != ""):
        return row[0]["content"]
    return None

def extract_answer(row):
    if (row[1]["role"] == "assistant" and row[1]["content"] != ""):
        return row[1]["content"]
    return None

df["question_a"] = df.apply(lambda row: extract_question(row["conversation_a"]), axis=1)
df["question_b"] = df.apply(lambda row: extract_question(row["conversation_b"]), axis=1)
df["answer_a"] = df.apply(lambda row: extract_answer(row["conversation_a"]), axis=1)
df["answer_b"] = df.apply(lambda row: extract_answer(row["conversation_b"]), axis=1)

df.dropna(subset=["question_a", "question_b", "answer_a", "answer_b"], inplace=True)

df = df[["model_a", "model_b", "winner", "question_a", "question_b", "answer_a", "answer_b"]]
df.head()


,model_a,model_b,winner,question_a,question_b,answer_a,answer_b
0,chatglm-6b,koala-13b,model_b,What is the difference between OpenCL and CUDA?,What is the difference between OpenCL and CUDA?,OpenCL and CUDA are two different programming ...,OpenCL and CUDA are both programming languages...
1,oasst-pythia-12b,alpaca-13b,tie,Why did my parent not invite me to their wedding?,Why did my parent not invite me to their wedding?,It is possible that your parent did not invite...,It is likely that they wanted to keep the gues...
2,koala-13b,oasst-pythia-12b,model_b,"Fuji vs. Nikon, which is better?","Fuji vs. Nikon, which is better?",Both Fuji and Nikon are popular camera brands ...,This is a subjective question and the answer d...
3,vicuna-13b,oasst-pythia-12b,model_b,How to build an arena for chatbots?,How to build an arena for chatbots?,Building an arena for chatbots can be done by ...,Building an arena for chatbots is a great way ...
4,vicuna-13b,koala-13b,model_a,When is it today?,When is it today?,"I'm sorry, I cannot determine the current date...","Today is February 23, 2023."


In [19]:
# drop if question_a != question_b

df = df[df["question_a"] == df["question_b"]]
df["question"] = df["question_a"]
df.drop(columns=["question_a", "question_b"], inplace=True)
df.head()

,model_a,model_b,winner,answer_a,answer_b,question
0,chatglm-6b,koala-13b,model_b,OpenCL and CUDA are two different programming ...,OpenCL and CUDA are both programming languages...,What is the difference between OpenCL and CUDA?
1,oasst-pythia-12b,alpaca-13b,tie,It is possible that your parent did not invite...,It is likely that they wanted to keep the gues...,Why did my parent not invite me to their wedding?
2,koala-13b,oasst-pythia-12b,model_b,Both Fuji and Nikon are popular camera brands ...,This is a subjective question and the answer d...,"Fuji vs. Nikon, which is better?"
3,vicuna-13b,oasst-pythia-12b,model_b,Building an arena for chatbots can be done by ...,Building an arena for chatbots is a great way ...,How to build an arena for chatbots?
4,vicuna-13b,koala-13b,model_a,"I'm sorry, I cannot determine the current date...","Today is February 23, 2023.",When is it today?


In [20]:
df

,model_a,model_b,winner,answer_a,answer_b,question
0,chatglm-6b,koala-13b,model_b,OpenCL and CUDA are two different programming ...,OpenCL and CUDA are both programming languages...,What is the difference between OpenCL and CUDA?
1,oasst-pythia-12b,alpaca-13b,tie,It is possible that your parent did not invite...,It is likely that they wanted to keep the gues...,Why did my parent not invite me to their wedding?
2,koala-13b,oasst-pythia-12b,model_b,Both Fuji and Nikon are popular camera brands ...,This is a subjective question and the answer d...,"Fuji vs. Nikon, which is better?"
3,vicuna-13b,oasst-pythia-12b,model_b,Building an arena for chatbots can be done by ...,Building an arena for chatbots is a great way ...,How to build an arena for chatbots?
4,vicuna-13b,koala-13b,model_a,"I'm sorry, I cannot determine the current date...","Today is February 23, 2023.",When is it today?
...,...,...,...,...,...,...
32994,palm-2,chatglm-6b,model_a,"A litre is equal to 1000 ml, so 50 litres is e...",To find out how long it will take to fill a co...,If a tap is outputting 2500 ml a minute. How m...
32995,alpaca-13b,claude-instant-v1,tie,Joe Biden is currently the President of the Un...,Joe Biden is currently the 46th president of t...,who is the president of the U.S.A?
32996,claude-instant-v1,guanaco-33b,model_a,Training Stable Diffusion models like LoRA req...,Lora is a machine learning model that is used ...,how to train lora for stable diffusion? explai...
32998,guanaco-33b,koala-13b,model_a,Evaluating a language model output involves se...,Evaluating the output of a language model invo...,how to evaluate a language model output?


In [22]:
df = df[(df["winner"] == "model_a") | (df["winner"] == "model_b")]
df = df[df["model_a"] != df["model_b"]]
df

,model_a,model_b,winner,answer_a,answer_b,question
0,chatglm-6b,koala-13b,model_b,OpenCL and CUDA are two different programming ...,OpenCL and CUDA are both programming languages...,What is the difference between OpenCL and CUDA?
2,koala-13b,oasst-pythia-12b,model_b,Both Fuji and Nikon are popular camera brands ...,This is a subjective question and the answer d...,"Fuji vs. Nikon, which is better?"
3,vicuna-13b,oasst-pythia-12b,model_b,Building an arena for chatbots can be done by ...,Building an arena for chatbots is a great way ...,How to build an arena for chatbots?
4,vicuna-13b,koala-13b,model_a,"I'm sorry, I cannot determine the current date...","Today is February 23, 2023.",When is it today?
5,vicuna-13b,koala-13b,model_a,"1, 4, 7, 10\n\nCounting with a step of 3 means...","1, 4, 7, 10",Count from 1 to 10 with step = 3
...,...,...,...,...,...,...
32990,palm-2,wizardlm-13b,model_a,JQ is a lightweight and flexible command-line ...,`jq` is a command line tool that allows you to...,what is the command line tool `jq`?
32993,claude-instant-v1,gpt4all-13b-snoozy,model_a,Here is a plot outline for a stoner comedy fil...,"Title: ""Cynthia's High Journey""\n\nPlot:\nCynt...",Describe a plot for a stoner comedy film set i...
32994,palm-2,chatglm-6b,model_a,"A litre is equal to 1000 ml, so 50 litres is e...",To find out how long it will take to fill a co...,If a tap is outputting 2500 ml a minute. How m...
32996,claude-instant-v1,guanaco-33b,model_a,Training Stable Diffusion models like LoRA req...,Lora is a machine learning model that is used ...,how to train lora for stable diffusion? explai...


In [23]:
df.to_csv("excluded/arena/arena.csv", index=False)

In [26]:
# Extract rows where model_a is vicuna and model_b is koala or vice versa
vicuna_koala_vs = df[(df["model_a"].str.contains("vicuna") & df["model_b"].str.contains("koala")) | 
                     (df["model_a"].str.contains("koala") & df["model_b"].str.contains("vicuna"))]

print(f"Found {len(vicuna_koala_vs)} comparisons between Vicuna and Koala models")
vicuna_koala_vs

Found 576 comparisons between Vicuna and Koala models


,model_a,model_b,winner,answer_a,answer_b,question
4,vicuna-13b,koala-13b,model_a,"I'm sorry, I cannot determine the current date...","Today is February 23, 2023.",When is it today?
5,vicuna-13b,koala-13b,model_a,"1, 4, 7, 10\n\nCounting with a step of 3 means...","1, 4, 7, 10",Count from 1 to 10 with step = 3
6,vicuna-13b,koala-13b,model_a,1. 💪 Sharing progress or achievements\n2. 📝 Sh...,1. Sharing hands\n* Share icon\n* Group ...,"Emoji for ""sharing"". List 10"
10,koala-13b,vicuna-13b,model_a,The future of bitcoin is uncertain and subject...,The future of bitcoin is uncertain and subject...,What is the future of bitcoin?
15,vicuna-13b,koala-13b,model_b,......,"A long time ago in a galaxy far, far away, the...","A long time ago in a galaxy far, far away"
...,...,...,...,...,...,...
31687,koala-13b,vicuna-13b,model_b,Hello! How can I help you today? Is there anyt...,Hello! How can I assist you today?,Hello
31770,koala-13b,vicuna-7b,model_b,It is difficult to say who the smartest man in...,It is difficult to determine who the smartest ...,Who is the smartest man in the world?
31995,vicuna-7b,koala-13b,model_b,(The opening shot is of Theo and Rachel sittin...,"Theo: Hey guys, welcome back to our channel. T...",Write a script for a YouTube video starring a ...
32286,koala-13b,vicuna-13b,model_a,"If you don't like old western movies, there ar...",There are many great movies out there that you...,I'm 14 years old and don't like the old wester...


In [29]:
vicuna_a = vicuna_koala_vs[vicuna_koala_vs["model_a"].str.contains("vicuna")]
vicuna_a

,model_a,model_b,winner,answer_a,answer_b,question
4,vicuna-13b,koala-13b,model_a,"I'm sorry, I cannot determine the current date...","Today is February 23, 2023.",When is it today?
5,vicuna-13b,koala-13b,model_a,"1, 4, 7, 10\n\nCounting with a step of 3 means...","1, 4, 7, 10",Count from 1 to 10 with step = 3
6,vicuna-13b,koala-13b,model_a,1. 💪 Sharing progress or achievements\n2. 📝 Sh...,1. Sharing hands\n* Share icon\n* Group ...,"Emoji for ""sharing"". List 10"
15,vicuna-13b,koala-13b,model_b,......,"A long time ago in a galaxy far, far away, the...","A long time ago in a galaxy far, far away"
19,vicuna-13b,koala-13b,model_a,Here is an example TypeScript function that ta...,<div><div></div></div>\n<div><div><div><div><p...,Write TypeScript function to produce full name...
...,...,...,...,...,...,...
30201,vicuna-13b,koala-13b,model_b,"As an AI language model, I don't have emotions...","As an AI language model, I don't have feelings...",Should an AI assistant like you “love” humanity?
30816,vicuna-7b,koala-13b,model_b,"Individuals have two X chromosomes, one inheri...",The average number of X chromosomes in a human...,What is the average number of X chromosomes?
30853,vicuna-7b,koala-13b,model_a,The Bessemer process was a major innovation in...,The Bessemer process was a significant improve...,Was the steel produced from the Bessemer proce...
31473,vicuna-7b,koala-13b,model_a,A language model,"I'm sorry, but I can't fulfill that request. C...","Roses are red, the sky is blue and your are..."


In [32]:
random_100 = vicuna_a.sample(n=100, random_state=31)
random_100

,model_a,model_b,winner,answer_a,answer_b,question
24817,vicuna-7b,koala-13b,model_a,It is generally not possible to solve a 2D bin...,"Yes, there is a fast way to solve a 2D bin-pac...",Is there a fast way to solve a 2D bin-packing ...
985,vicuna-13b,koala-13b,model_a,Bike fitting is the process of adjusting the p...,A bike fitting is a process in which an indivi...,What is done when bike fitting?
1029,vicuna-13b,koala-13b,model_b,"Yes, you are correct. LLM stands for Large Lan...",I apologize for the confusion. I see that LLM ...,"No, LLM stand for Large Language Models. This ..."
531,vicuna-13b,koala-13b,model_a,"A parsec is a unit of distance in astronomy, e...",A parsec is a unit of distance in astronomy th...,How many furlongs in a parsec?
406,vicuna-13b,koala-13b,model_a,Thought: I need to perform a mathematical calc...,Thought: To calculate the product of 258 and 5...,Answer the following questions as best you can...
...,...,...,...,...,...,...
759,vicuna-13b,koala-13b,model_b,"Yes, the claim is fully faithful to the docume...",It is not possible to say whether the claim is...,"Given the next claim and document, answer in y..."
497,vicuna-13b,koala-13b,model_b,Sandhyavandhanam is a Sanskrit term that is us...,Sandhyavandhanam is a Hindu ritual performed i...,What is the meaning of Sandhyavandhanam?
21599,vicuna-7b,koala-13b,model_a,1. The Legend of Zelda: Breath of the Wild and...,"1. ""Dark Souls: A Game of Thrones""\n2. ""Th...",We have two different things: the videogame Th...
861,vicuna-13b,koala-13b,model_a,In the ocean's depths\nA silvery fish glides b...,"Fish swims in sea,Mysterious and free.",Write a haiku about a fish


In [25]:
random_200 = df.sample(n=200, random_state=42)
random_200

,model_a,model_b,winner,answer_a,answer_b,question
23607,vicuna-7b,RWKV-4-Raven-14B,model_b,"Yes, I am familiar with the Hooke-Jeeves patte...","Yes, I am familiar with the Hooke-Jeeves patte...",Are you familiar with the Hooke-Jeeves pattern...
15636,palm-2,vicuna-13b,model_b,The Byzantine Generals Algorithm (BGA) is a fa...,The Byzantine Generals Algorithm is a thought ...,What is the byzantine generals algorithm?
25402,chatglm-6b,gpt-3.5-turbo,model_a,Nginx configuration is a set of files that def...,"To write an Nginx configuration file, follow t...",how to write a nginx config?
3808,oasst-pythia-12b,koala-13b,model_b,You are correct that the two queries can be co...,You can combine the two queries into a single ...,var files = from f in _context.ExpenseImportFi...
26544,koala-13b,claude-instant-v1,model_b,"The dosage of Polapola, Ceylon Leadwort (Aerva...",I would not recommend taking Polapola/Aerva la...,"what should the dosage be for using Polapola, ..."
...,...,...,...,...,...,...
1793,oasst-pythia-12b,stablelm-tuned-alpha-7b,model_a,Inositol is a natural substance found in the b...,Inositol is an important natural source of glu...,how does inositol improve sleep?
18697,stablelm-tuned-alpha-7b,alpaca-13b,model_a,"In CakePHP 3, you can use the `URL::asset()` m...","Sure. In CakePHP 3, you can obtain the full UR...",can tell me how we get complate url in cakephp3
32273,oasst-pythia-12b,alpaca-13b,model_b,"The song ""In My Life"" by the Beatles includes ...","The song ""She's Leaving Home"" by The Beatles f...",I would like to have a complete description of...
2419,chatglm-6b,koala-13b,model_b,Sure! Here's a budget-friendly itinerary for 5...,"Sure, here's a low-cost 5-night vacation in Ba...","make me a vacation in Bali for 5 nights, low cost"


In [33]:
random_200.to_csv("datasets/arena/random_200.csv", index=False)

In [34]:
random_100.to_csv("datasets/arena/vicukoala_100.csv", index=False)

In [35]:
# Create a copy of the dataframe to work with
swapped_df = random_100.copy()

# Swap model_a and model_b
swapped_df["temp"] = swapped_df["model_a"]
swapped_df["model_a"] = swapped_df["model_b"]
swapped_df["model_b"] = swapped_df["temp"]
swapped_df.drop(columns=["temp"], inplace=True)

# Swap answer_a and answer_b
swapped_df["temp"] = swapped_df["answer_a"]
swapped_df["answer_a"] = swapped_df["answer_b"]
swapped_df["answer_b"] = swapped_df["temp"]
swapped_df.drop(columns=["temp"], inplace=True)

# Swap winner values
swapped_df["winner"] = swapped_df["winner"].apply(lambda x: "model_b" if x == "model_a" else "model_a")

# Display the first few rows to verify the swap
swapped_df

,model_a,model_b,winner,answer_a,answer_b,question
24817,koala-13b,vicuna-7b,model_b,"Yes, there is a fast way to solve a 2D bin-pac...",It is generally not possible to solve a 2D bin...,Is there a fast way to solve a 2D bin-packing ...
985,koala-13b,vicuna-13b,model_b,A bike fitting is a process in which an indivi...,Bike fitting is the process of adjusting the p...,What is done when bike fitting?
1029,koala-13b,vicuna-13b,model_a,I apologize for the confusion. I see that LLM ...,"Yes, you are correct. LLM stands for Large Lan...","No, LLM stand for Large Language Models. This ..."
531,koala-13b,vicuna-13b,model_b,A parsec is a unit of distance in astronomy th...,"A parsec is a unit of distance in astronomy, e...",How many furlongs in a parsec?
406,koala-13b,vicuna-13b,model_b,Thought: To calculate the product of 258 and 5...,Thought: I need to perform a mathematical calc...,Answer the following questions as best you can...
...,...,...,...,...,...,...
759,koala-13b,vicuna-13b,model_a,It is not possible to say whether the claim is...,"Yes, the claim is fully faithful to the docume...","Given the next claim and document, answer in y..."
497,koala-13b,vicuna-13b,model_a,Sandhyavandhanam is a Hindu ritual performed i...,Sandhyavandhanam is a Sanskrit term that is us...,What is the meaning of Sandhyavandhanam?
21599,koala-13b,vicuna-7b,model_b,"1. ""Dark Souls: A Game of Thrones""\n2. ""Th...",1. The Legend of Zelda: Breath of the Wild and...,We have two different things: the videogame Th...
861,koala-13b,vicuna-13b,model_b,"Fish swims in sea,Mysterious and free.",In the ocean's depths\nA silvery fish glides b...,Write a haiku about a fish


In [36]:
swapped_df.to_csv("datasets/arena/vicukoala_100_swapped.csv", index=False)